In [1]:
from pathlib import Path
import re
import requests
import time
import datetime

import pandas as pd
from requests_html import HTML
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [2]:
BASE_DIR = Path.cwd()
DATA_DIR = BASE_DIR / "data" # os.path.join(BASE_DIR, 'data')
if not DATA_DIR.exists(): # os.path.exists(DATA_DIR)
    DATA_DIR.mkdir(exist_ok=True) # os.makedirs(DATA_DIR, exist_ok=True)
    
product_category_links_output = DATA_DIR / "category-products.csv"
product_output = DATA_DIR / "products.csv"

In [3]:
options = Options()
options.add_argument("--headless")

driver = webdriver.Chrome(options=options)

In [4]:
categories = [
    {"name": "toys-and-games", "url": "https://www.amazon.com/Best-Sellers-Toys-Games/zgbs/toys-and-games/"},
    {"name": "electronics", "url": "https://www.amazon.com/Best-Sellers-Electronics/zgbs/electronics/"},
    {"name": "fashion", "url": "https://www.amazon.com/Best-Sellers/zgbs/fashion/"}
]

In [5]:
regex_options = [
    r"https://www.amazon.com/gp/product/(?P<product_id>[\w-]+)/",
    r"https://www.amazon.com/dp/(?P<product_id>[\w-]+)/",
    r"https://www.amazon.com/(?P<slug>[\w-]+)/dp/(?P<product_id>[\w-]+)/",
]

def extract_product_id_from_url(url):
    product_id = None
    for regex_str in regex_options:
        regex = re.compile(regex_str)
        match = regex.match(url)
        if match != None:
            try:
                product_id = match['product_id']
            except:
                pass
    return product_id

In [6]:
def clean_page_links(page_links=[], category=None):
    final_page_links = []
    for url in page_links:
        product_id = extract_product_id_from_url(url)
        if product_id != None:
            final_page_links.append({"url": url, "product_id": product_id, "category": category})
    return final_page_links

In [7]:
def scrace_category_product_links(categories=[]):
    all_product_links = []
    for category in categories:
        time.sleep(1.5)
        url = category.get("url")
        driver.get(url)
        body_el = driver.find_element_by_css_selector("body")
        html_str = body_el.get_attribute("innerHTML")
        html_obj = HTML(html=html_str)
        page_links = [f"https://www.amazon.com{x}" for x in html_obj.links if x.startswith("/")]
        cleaned_links = clean_page_links(page_links=page_links, category=category)
        all_product_links += cleaned_links
    return all_product_links

In [8]:
def extract_categories_and_save(categories=[]):
    all_product_links = scrace_category_product_links(categories)
    category_df = pd.DataFrame(all_product_links)
    category_df.to_csv(product_category_links_output, index=False)

In [9]:
extract_categories_and_save(categories=categories)

In [10]:
def scrape_product_page(url, title_lookup = "#productTitle", price_lookup = "#priceblock_ourprice"):
    driver.get(url)
    time.sleep(1.5)
    body_el = driver.find_element_by_css_selector("body")
    html_str = body_el.get_attribute("innerHTML")
    html_obj = HTML(html=html_str)
    product_title = html_obj.find(title_lookup, first=True).text
    product_price = html_obj.find(price_lookup, first=True).text
    return product_title, product_price

In [11]:
def perform_scrape(cleaned_items=[]):
    data_extracted = []
    for obj in cleaned_items:
        link = obj['url']
        product_id = obj['product_id']
        title, price = (None, None)
        try:
            title, price = scrape_product_page(link)
        except:
            pass
        if title != None and price != None:
            print(link, title, price)
        product_data = {
            "url": link,
            "product_id": product_id,
            "title": title,
            "price": price
        }
        data_extracted.append(product_data)
    return data_extracted

In [12]:
def row_scrape_event(row, *args, **kwargs):
    link = row['url']
    scraped = 0
    try:
        scraped = row['scraped']
    except:
        pass
    # print(link)
    if scraped == 1 or scraped == "1":
        print("skipped")
        return row
    product_id = row['product_id']
    title, price = (None, None)
    try:
        title, price = scrape_product_page(link)
    except:
        pass
    row['title'] = title
    row['price'] = price
    row['scraped'] = 1
    row['timestamp'] = datetime.datetime.now().timestamp()
    print(link, title, price)
    return row

In [13]:
df = pd.read_csv(product_category_links_output)
df.head()

,url,product_id,category
0,https://www.amazon.com/Throw-Burrito-Dodgeball...,B07TS96J7Q,"{'name': 'toys-and-games', 'url': 'https://www..."
1,https://www.amazon.com/Disney-Pixar-Story-Alie...,B07HD27YZT,"{'name': 'toys-and-games', 'url': 'https://www..."
2,https://www.amazon.com/Paw-Patrol-Cruiser-Vehi...,B07PTSZ43T,"{'name': 'toys-and-games', 'url': 'https://www..."
3,https://www.amazon.com/Flybar-Jumper-Toddlers-...,B00WTDXSDM,"{'name': 'toys-and-games', 'url': 'https://www..."
4,https://www.amazon.com/Pacon-PAC6555-Lightweig...,B0013CDJTS,"{'name': 'toys-and-games', 'url': 'https://www..."


In [15]:
df_sub = df.copy() # df.head(n=10)

In [16]:
df_sub = df_sub.apply(row_scrape_event, axis=1)

https://www.amazon.com/Throw-Burrito-Dodgeball-Exploding-Kittens/dp/B07TS96J7Q/ref=zg_bs_toys-and-games_46/130-1176424-8487930?_encoding=UTF8&psc=1&refRID=95PB7KCZPW5HP194TFCW Throw Throw Burrito by Exploding Kittens - A Dodgeball Card Game - Family-Friendly Party Games - Card Games for Adults, Teens & Kids $19.99
https://www.amazon.com/Disney-Pixar-Story-Aliens-Figure/dp/B07HD27YZT/ref=zg_bs_toys-and-games_42/130-1176424-8487930?_encoding=UTF8&psc=1&refRID=95PB7KCZPW5HP194TFCW Disney Pixar Toy Story Space Aliens Figures $12.00
https://www.amazon.com/Paw-Patrol-Cruiser-Vehicle-Collectible/dp/B07PTSZ43T/ref=zg_bs_toys-and-games_25/130-1176424-8487930?_encoding=UTF8&psc=1&refRID=95PB7KCZPW5HP194TFCW Paw Patrol, Chase’s Patrol Cruiser Vehicle with Collectible Figure, for Kids Aged 3 and Up $9.99
https://www.amazon.com/Flybar-Jumper-Toddlers-Durable-Supports/dp/B00WTDXSDM/ref=zg_bs_toys-and-games_33/130-1176424-8487930?_encoding=UTF8&psc=1&refRID=95PB7KCZPW5HP194TFCW Flybar My First Foam P

https://www.amazon.com/Crayola-Colored-Pre-sharpened-Coloring-Stocking/dp/B018HB2QFU/ref=zg_bs_toys-and-games_34/130-1176424-8487930?_encoding=UTF8&psc=1&refRID=95PB7KCZPW5HP194TFCW None None
https://www.amazon.com/Sassy-Stacks-Circles-Stacking-Learning/dp/B07NXDJ52C/ref=zg_bs_toys-and-games_29/130-1176424-8487930?_encoding=UTF8&psc=1&refRID=95PB7KCZPW5HP194TFCW Sassy Stacks of Circles Stacking Ring STEM Learning Toy, 9 Piece Set, Age 6+ Months $8.99
https://www.amazon.com/First-Years-Stack-Up-Cups/dp/B00005C5H4/ref=zg_bs_toys-and-games_7/130-1176424-8487930?_encoding=UTF8&psc=1&refRID=95PB7KCZPW5HP194TFCW The First Years Stack Up Cup Toys $3.99
https://www.amazon.com/Mattel-Games-Official-Amazon-Exclusive/dp/B07P6MZPK3/ref=zg_bs_toys-and-games_22/130-1176424-8487930?_encoding=UTF8&psc=1&refRID=95PB7KCZPW5HP194TFCW Mattel Games UNO $9.99
https://www.amazon.com/SunWorks-Construction-Paper-Black-Sheets/dp/B002LARR7Q/ref=zg_bs_toys-and-games_12/130-1176424-8487930?_encoding=UTF8&psc=1&ref

https://www.amazon.com/Roku-Premiere-HDR-Streaming-Player-Premium/dp/B07HDBZN7Q/ref=zg_bs_electronics_46?_encoding=UTF8&psc=1&refRID=PPBEXGXKST3134BKXST7 Roku Premiere | HD/4K/HDR Streaming Media Player, Simple Remote and Premium HDMI Cable $39.00
https://www.amazon.com/OHill-Organizer-Management-Organizing-Adhesive/dp/B071FXZBMV/ref=zg_bs_electronics_21?_encoding=UTF8&psc=1&refRID=PPBEXGXKST3134BKXST7 None None
https://www.amazon.com/Roku-Express-Streaming-Media-Player/dp/B07WVFCVJN/ref=zg_bs_electronics_12?_encoding=UTF8&psc=1&refRID=PPBEXGXKST3134BKXST7 Roku Express HD Streaming Media Player 2019 $29.00
https://www.amazon.com/Sceptre-E205W-16003R-Frameless-Speakers-Metallic/dp/B07743412C/ref=zg_bs_electronics_43?_encoding=UTF8&psc=1&refRID=PPBEXGXKST3134BKXST7 Sceptre 20" 1600x900 75Hz Ultra Thin LED Monitor 2x HDMI VGA Built-in Speakers, Machine Black Wide Viewing Angle 170° (Horizontal) / 160° (Vertical) $74.97
https://www.amazon.com/Fujifilm-INSTAX-Instant-Twin-Pack/dp/B00EB4ADQW

https://www.amazon.com/Revolution-Cherokee-Moderate-Drawstring-XX-Small/dp/B071GPNK7R/ref=zg_bs_fashion_23?_encoding=UTF8&psc=1&refRID=6KBTVSJ56041PF82TJ0M None None
https://www.amazon.com/Womens-Water-Barefoot-Quick-Dry-Exercise/dp/B07BPZ8S9W/ref=zg_bs_fashion_37?_encoding=UTF8&psc=1&refRID=6KBTVSJ56041PF82TJ0M None None
https://www.amazon.com/MIHOLL-Womens-Summer-Casual-Sleeve/dp/B07N4P8VM9/ref=zg_bs_fashion_40?_encoding=UTF8&psc=1&refRID=6KBTVSJ56041PF82TJ0M None None
https://www.amazon.com/Urban-CoCo-Womens-Versatile-Stretchy/dp/B06WD5Z6SJ/ref=zg_bs_fashion_47?_encoding=UTF8&psc=1&refRID=6KBTVSJ56041PF82TJ0M None None
https://www.amazon.com/Hanes-Full-Zip-EcoSmart-Fleece-Hoodie/dp/B071JDWV4R/ref=zg_bs_fashion_31?_encoding=UTF8&psc=1&refRID=6KBTVSJ56041PF82TJ0M None None
https://www.amazon.com/Champion-Jersey-Short-Pockets-Black/dp/B00AKSCXBQ/ref=zg_bs_fashion_27?_encoding=UTF8&psc=1&refRID=6KBTVSJ56041PF82TJ0M None None
https://www.amazon.com/BALEAF-Womens-Workout-Control-Pockets/d

In [17]:
# df.to_csv(product_output, index=False)

In [18]:
products_df = pd.read_csv(product_output)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/elfi.yang@trussle.com/Documents/30-day/price-track-and-monitor/notebooks/data/products.csv'

In [ ]:
final_df = pd.concat([products_df, df_sub])
final_df.to_csv(product_output, index=False)

In [ ]:
final_df.head()